In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
COLUMNS = ["age", "workclass", "edu_level",
           "marital_status", "occupation", "relationship",
           "race", "sex", "hours_per_week",
           "native_country", "income"]

train_df = pd.read_csv(
    filepath_or_buffer="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    names=COLUMNS,
    engine='python',
    usecols=[0, 1, 4, 5, 6, 7, 8, 9, 12, 13, 14],
    sep=r'\s*,\s*',
    na_values="?"
)

test_df = pd.read_csv(
    filepath_or_buffer="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    names=COLUMNS,
    skiprows=[0],
    engine='python',
    usecols=[0, 1, 4, 5, 6, 7, 8, 9, 12, 13, 14],
    sep=r'\s*,\s*',
    na_values="?"
)


In [3]:
# Drop rows with missing values
train_df = train_df.dropna(how="any", axis=0)
test_df = test_df.dropna(how="any", axis=0)

# To reduce the complexity, we binarize the attribute
# To reduce the complexity, we binarize the attribute


def mapping(tuple):
    # age, 37
    tuple['age'] = 1 if tuple['age'] > 37 else 0
    # workclass
    tuple['workclass'] = 0 if tuple['workclass'] != 'Private' else 1
    # edu-level
    tuple['edu_level'] = 1 if tuple['edu_level'] > 9 else 0
    # maritial statue
    tuple['marital_status'] = 1 if tuple['marital_status'] == "Married-civ-spouse" else 0
    # occupation
    tuple['occupation'] = 1 if tuple['occupation'] == "Craft-repair" else 0
    # relationship
    tuple['relationship'] = 0 if tuple['relationship'] == "Not-in-family" else 1
    # race
    tuple['race'] = 0 if tuple['race'] != "White" else 1
    # sex
    tuple['sex'] = 0 if tuple['sex'] != "Male" else 1
    # hours per week
    tuple['hours_per_week'] = 1 if tuple['hours_per_week'] > 40 else 0
    # native country
    tuple['native_country'] = 1 if tuple['native_country'] == "United-States" else 0
    # income
    tuple['income'] = 1 if tuple['income'] == '>50K' or tuple['income'] == '>50K.' else 0
    return tuple


train_df = train_df.apply(mapping, axis=1)
test_df = test_df.apply(mapping, axis=1)

In [65]:
train_df[0:30]

,age,workclass,edu_level,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
0,1,0,1,0,0,0,1,1,0,1,0
1,1,0,1,1,0,1,1,1,0,1,0
2,1,1,0,0,0,0,1,1,0,1,0
3,1,1,0,1,0,1,0,1,0,1,0
4,0,1,1,1,0,1,0,0,0,0,0
5,0,1,1,1,0,1,1,0,0,1,0
6,1,1,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,1,1,1,1,1,1
8,0,1,1,0,0,0,1,0,1,1,1
9,1,1,1,1,0,1,1,1,0,1,1


In [5]:
train_data = torch.from_numpy(train_df.values)
test_data = torch.from_numpy(test_df.values)
# merge two datasets
dataset = torch.cat((train_data,test_data), 0)

In [6]:
from torch.utils.data import Dataset,DataLoader
class AdultDataset(Dataset):
    def __init__(self, data_set):
        self.x = data_set
        self.len = data_set.size()[0]
    def __getitem__(self,index):
        return self.x[index]
    def __len__(self):
        return self.len
adultDataset = AdultDataset(dataset)
dataLoader = DataLoader(dataset=adultDataset, batch_size=128, shuffle=True)



In [149]:
# a basic Generator


class Generator(nn.Module):
    def __init__(self, f, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size*2)
        self.map2 = nn.Linear(hidden_size*2, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        # f is action function
        self.f = f

    def forward(self, x):
        x = self.map1(x)
        x = self.f(x)
        x = self.map2(x)
        x = self.f(x)
        x = self.map3(x)
        x = self.f(x)
        return x

# a basic Discriminator

In [150]:
class Discriminator(nn.Module):
    def __init__(self, f, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        # f is action function
        self.f = f

    def forward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        return torch.sigmoid(self.map3(x))

In [185]:
class CFGAN(nn.Module):
    def __init__(self, f):
        super(CFGAN, self).__init__()

        self.age_net = Generator(
            f, 1, 4, 1)
        self.workclass_net = Generator(
            f, 5, 8, 1)
        self.edu_level_net = Generator(
            f, 6, 16, 1)
        self.marital_status_net = Generator(
            f, 5, 8, 1)
        self.occupation_net = Generator(
            f, 6, 16, 1)
        self.relationship_net = Generator(
            f, 6, 16, 1)
        self.race_net = Generator(
            f, 1, 4, 1)
        self.sex_net = Generator(
            f, 1, 4, 1)
        self.hours_per_week_net = Generator(
            f, 7, 16, 1)
        self.native_country_net = Generator(
            f, 1, 4, 1)
        self.income_net = Generator(
            f, 11, 32, 1)

    def forward(self, input, intervention=-1):
        name = ["age","workclass","edu_level","marital_status","occupation","relationship","race","sex","hours_per_week","native_country","income"]
        Z = dict(zip(name, input.transpose(0, 1).view(len(name), -1, 1)))

        # hight = 0 in the graph
        # sex should considered about intervention
        if(intervention == -1):
            self.sex = self.race_net(Z["sex"])
        elif(intervention == 0):
            self.sex = torch.zeros(Z["sex"].size())
        else:
            self.sex = torch.ones(Z["sex"].size())
        self.age = self.age_net(Z["age"])
        self.race = self.sex_net(Z["race"])
        self.native_country = self.native_country_net(Z["native_country"])

        # hight = 1 in the graph
        self.marital_status = self.marital_status_net(torch.cat(
            [Z["marital_status"], self.race, self.age,
                self.sex, self.native_country], 1
        ))

        # hight = 2 in the gragh
        self.edu_level = self.edu_level_net(torch.cat(
            [Z["edu_level"], self.race, self.age, self.sex,
                self.native_country, self.marital_status], 1
        ))

        # hight = 3 in the gragh
        self.occupation = self.occupation_net(torch.cat(
            [Z["occupation"], self.race, self.age, self.sex,
                self.marital_status, self.edu_level], 1
        ))

        self.hours_per_week = self.hours_per_week_net(torch.cat(
            [Z["hours_per_week"], self.race, self.age, self.sex,
             self.native_country, self.marital_status, self.edu_level], 1
        ))

        self.workclass = self.workclass_net(torch.cat(
            [Z["workclass"], self.age, self.marital_status,
                self.edu_level, self.native_country], 1
        ))

        self.relationship = self.relationship_net(torch.cat(
            [Z["relationship"], self.age, self.sex, self.native_country,
                self.marital_status, self.edu_level], 1
        ))

        # hight = 4 in the gragh

        self.income = self.income_net(torch.cat(
            [Z["income"], self.race, self.age, self.sex, self.native_country, self.marital_status,
                self.edu_level, self.occupation, self.hours_per_week, self.workclass, self.relationship], 1
        ))

        return torch.cat([self.age, self.workclass, self.edu_level, self.marital_status,
        self.occupation, self.relationship, self.race, self.sex,
        self.hours_per_week, self.native_country, self.income], 1)

In [186]:
num_epochs = 100
g_steps = 5
g2_steps = 50
batch = 128
LR = 0.001
print_interval = 1
# action function
discriminator_activation_function = nn.LeakyReLU(0.2)
generator_activation_function = torch.tanh

# net init
discriminator_1 = Discriminator(
    discriminator_activation_function, 11, 64, 1)
generator = CFGAN(generator_activation_function)

# Binary cross entropy: https://pytorch.org/docs/stable/nn.html?highlight=bceloss#torch.nn.BCELoss
criterion = nn.BCELoss()
# optim
generator_optim = torch.optim.Adam(
    generator.parameters(), lr=LR, betas=(0.9, 0.99))
discriminator_1_optim = torch.optim.Adam(
    discriminator_1.parameters(), lr=LR, betas=(0.9, 0.99))



In [13]:
import copy

# debug
## test for paramaters

In [217]:
data = copy.copy(dataLoader)
for real_data in data:

    # 1A: Train D1 on real
    discriminator_1.zero_grad()
    d_real_data = real_data
    # real data's lable should be true
    d_real_labe = torch.ones(d_real_data.size()[0])
    d_real_decision = discriminator_1(d_real_data.float())
    d_real_loss = criterion(
        torch.squeeze(d_real_decision), d_real_labe)
    d_real_loss.backward()

    # 1B: Train D1 on fake data
    d_fake_data = generator(torch.randn(batch, 11))
    # print(d_fake_data.size())
    d_fake_lable = torch.zeros(batch)
    d_fake_decision = discriminator_1(d_fake_data)
    d_fake_loss = criterion(torch.squeeze(
        d_fake_decision), d_fake_lable)
    d_fake_loss.backward()
    # Only optimizes D1's parameters
    discriminator_1_optim.step()

In [223]:

for i in range(20):
    data = copy.copy(dataLoader)
    for real_data in data:

        # 1A: Train D1 on real
        discriminator_1.zero_grad()
        d_real_data = real_data
        # real data's lable should be true
        d_real_labe = torch.ones(d_real_data.size()[0])
        d_real_decision = discriminator_1(d_real_data.float())
        d_real_loss = criterion(
            torch.squeeze(d_real_decision), d_real_labe)
        d_real_loss.backward()

        # 1B: Train D1 on fake data
        d_fake_data = generator(torch.randn(batch, 11))
        # print(d_fake_data.size())
        d_fake_lable = torch.zeros(batch)
        d_fake_decision = discriminator_1(d_fake_data)
        d_fake_loss = criterion(torch.squeeze(
            d_fake_decision), d_fake_lable)
        d_fake_loss.backward()
        # Only optimizes D1's parameters
        discriminator_1_optim.step()
    
    for g_index in range(64):
        # Train G on D's response
        generator.zero_grad()
        g_fake_data = generator(torch.randn(batch, 11))
        d_g_fake_decision = discriminator_1(g_fake_data)
        g_fake_lable = torch.ones(batch)
        g_loss = criterion(torch.squeeze(d_g_fake_decision), g_fake_lable)
        g_loss.backward()
        generator_optim.step()
        
    print(i,d_real_loss.tolist(),d_fake_loss.tolist(),g_loss.tolist())

0 0.013045307248830795 0.0016629857709631324 0.01813497208058834
1 0.006974454037845135 0.0025961659848690033 0.019351253286004066
2 0.004128933884203434 0.0017923510167747736 0.013094463385641575
3 0.001056463923305273 0.019427934661507607 0.012560836970806122
4 0.00010539229697315022 0.015412031672894955 0.019025545567274094
5 0.003513236530125141 0.0014881686074659228 0.024885153397917747
6 0.012492464855313301 0.0033049280755221844 0.033149391412734985
7 0.0013619082747027278 0.00015893984527792782 0.008169825188815594
8 0.0036359746009111404 0.001317936577834189 0.044445864856243134
9 0.0013813667465001345 0.00036774633917957544 0.004945330787450075
10 0.013553413562476635 0.013049769215285778 0.02837757207453251
11 0.00222163088619709 0.0010353690013289452 0.008836967870593071
12 0.046411048620939255 0.005799140315502882 0.05417352542281151
13 0.00038549452438019216 0.0002163913013646379 0.004485161043703556
14 0.0003284133563283831 0.004493562504649162 0.03258446976542473
15 0.0

In [224]:
f = generator(torch.randn(20, 11))
print(f)
print(f.ge(0.5).long())
dis = discriminator_1(generator(torch.randn(batch, 11)))
dis

tensor([[ 0.9946,  1.0000, -0.1201,  0.9965,  0.9896,  1.0000,  0.9984,  1.0000,
         -0.2794,  0.9958,  0.0368],
        [ 0.0286,  1.0000, -0.1244,  0.9952,  0.9978,  1.0000,  0.9987,  1.0000,
         -0.2816,  0.9958,  0.0405],
        [ 0.9946,  1.0000,  0.8672, -0.0144,  0.9060,  1.0000,  0.9986,  1.0000,
         -0.2575,  0.9956,  0.0565],
        [ 0.9931,  1.0000, -0.1150,  0.9967,  0.9976,  1.0000,  0.9990,  0.9999,
         -0.3018,  0.9961,  0.0686],
        [ 0.0141,  1.0000,  0.9739,  0.0398,  0.9966,  0.9873,  0.9991,  0.9999,
         -0.3041,  0.9959,  0.0626],
        [ 0.9948,  1.0000, -0.1212,  0.8798,  0.9978,  1.0000,  0.9975,  1.0000,
         -0.2885,  0.9957,  0.0496],
        [ 0.9947,  1.0000,  0.9812,  0.9964,  0.9977,  1.0000,  0.9991,  1.0000,
         -0.2805,  0.9959,  0.1102],
        [ 0.9945,  1.0000, -0.1199,  0.9964,  0.9969,  1.0000,  0.9992,  1.0000,
         -0.2950,  0.9959,  0.0591],
        [ 0.9930,  1.0000,  0.9825, -0.0120,  0.9976,  1

tensor([[0.9993],
        [0.9894],
        [0.9987],
        [0.9810],
        [0.9668],
        [0.9337],
        [0.9931],
        [0.9984],
        [0.9879],
        [0.9943],
        [0.9522],
        [0.9864],
        [0.9685],
        [0.9943],
        [0.9969],
        [0.9986],
        [0.9935],
        [0.9988],
        [0.9981],
        [0.9996],
        [0.9996],
        [0.9997],
        [0.9946],
        [0.9631],
        [0.9949],
        [0.9805],
        [0.9974],
        [0.9969],
        [0.9868],
        [0.9992],
        [0.9575],
        [0.9792],
        [0.9777],
        [0.9461],
        [0.9497],
        [0.9941],
        [0.9995],
        [0.9888],
        [0.9929],
        [0.9996],
        [0.9934],
        [0.9969],
        [0.9999],
        [0.9750],
        [0.9804],
        [0.9995],
        [0.9992],
        [0.9998],
        [0.9383],
        [0.9938],
        [0.9941],
        [0.9589],
        [0.9681],
        [0.9985],
        [0.9931],
        [0

In [219]:
 for g_index in range(32):
    # Train G on D's response
    generator.zero_grad()
    g_fake_data = generator(torch.randn(batch, 11))
    d_g_fake_decision = discriminator_1(g_fake_data)
    g_fake_lable = torch.ones(batch)
    g_loss = criterion(torch.squeeze(d_g_fake_decision), g_fake_lable)
    g_loss.backward()
    generator_optim.step()

In [21]:
import copy
for epoch in range(num_epochs):
    # GAN1
    # 1. Train D on real+fake
    # D.zero_grad()
    data = copy.copy(dataLoader)

    for real_data in data:

        # 1A: Train D1 on real
        discriminator_1.zero_grad()
        d_real_data = real_data
        # real data's lable should be true
        d_real_labe = torch.ones(d_real_data.size()[0])
        d_real_decision = discriminator_1(d_real_data.float())
        d_real_loss = criterion(
            torch.squeeze(d_real_decision), d_real_labe)
        d_real_loss.backward()

        # 1B: Train D1 on fake data
        d_fake_data = generator(torch.randn(batch, 11))
        # print(d_fake_data.size())
        d_fake_lable = torch.zeros(batch)
        d_fake_decision = discriminator_1(d_fake_data)
        d_fake_loss = criterion(torch.squeeze(
            d_fake_decision), d_fake_lable)
        d_fake_loss.backward()
        # Only optimizes D1's parameters
        discriminator_1_optim.step()

        drl, dfl = d_real_loss.tolist(), d_fake_loss.tolist()
    for g_index in range(g_steps):
        # Train G on D's response
        generator.zero_grad()

        g_fake_data = generator(torch.randn(batch, 11))
        d_g_fake_decision = discriminator_1(g_fake_data)
        g_fake_lable = torch.ones(batch)
        g_loss = criterion(torch.squeeze(d_g_fake_decision), g_fake_lable)
        g_loss.backward()
        generator_optim.step()
        gl = g_loss.tolist()

    # GAN2
    for g_index in range(g2_steps):
        generator.zero_grad()
        noise_z = torch.randn(batch, 11)
        fake_data = generator(noise_z)
        # O = {race; native country}:(0,0) (0,1) (1,0) (1,1)
        noise_o0 = []
        noise_o1 = []
        noise_o2 = []
        noise_o3 = []

        for index, single_data in enumerate(fake_data):
            if(single_data[7] < 0.5 and single_data[9] < 0.5):
                noise_o0.append(noise_z[index].view(1, -1))
            elif(single_data[7] < 0.5 and single_data[9] >= 0.5):
                noise_o1.append(noise_z[index].view(1, -1))
            elif(single_data[7] >= 0.5 and single_data[9] < 0.5):
                noise_o2.append(noise_z[index].view(1, -1))
            else:
                noise_o3.append(noise_z[index].view(1, -1))
        ge0,ge1,ge2,ge3 = None,None,None,None
        if(len(noise_o0) != 0):
            noise_o0 = torch.cat(noise_o0)
            o0_0_lable = generator(noise_o0, 0)[:, -1]
            o0_1_lable = generator(noise_o0, 1)[:, -1].detach()
            g_error0 = criterion(o0_0_lable, o0_1_lable)
            g_error0.backward()
            ge0 = g_error0.tolist()
        if(len(noise_o1) != 0):
            noise_o1 = torch.cat(noise_o1)
            o1_0_lable = generator(noise_o1, 0)[:, -1]
            o1_1_lable = generator(noise_o1, 1)[:, -1].detach()
            g_error1 = criterion(o1_0_lable, o1_1_lable)
            g_error1.backward()
            ge1 = g_error1.tolist()
        if(len(noise_o2) != 0):
            noise_o2 = torch.cat(noise_o2)
            o2_0_lable = generator(noise_o2, 0)[:, -1]
            o2_1_lable = generator(noise_o2, 1)[:, -1].detach()
            g_error2 = criterion(o2_0_lable, o2_1_lable)
            g_error2.backward()
            ge2 = g_error2.tolist()
        if(len(noise_o3) != 0):
            noise_o3 = torch.cat(noise_o3)
            o3_0_lable = generator(noise_o3, 0)[:, -1]
            o3_1_lable = generator(noise_o3, 1)[:, -1].detach()
            g_error3 = criterion(o3_0_lable, o3_1_lable)
            g_error3.backward()
            ge3 = g_error3.tolist()

        generator_optim.step()

    if epoch % print_interval == 0:
        print("Epoch %s: D (%s real_err, %s fake_err) G_l (%s err) G_0l (%s) G_1l (%s) G_2l (%s) G_3l (%s);" % (
            epoch, drl, dfl, gl, ge0, ge1, ge2, ge3))

Epoch 0: D (0.22125402092933655 real_err, 0.059217192232608795 fake_err) G_l (0.015695534646511078 err) G_0l (None) G_1l (None) G_2l (0.07190127670764923) G_3l (0.07072243094444275);
Epoch 1: D (0.8784480094909668 real_err, 0.14986123144626617 fake_err) G_l (0.05311468243598938 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.04634556174278259);
Epoch 2: D (0.32248008251190186 real_err, 0.18921847641468048 fake_err) G_l (0.010749738663434982 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.06491663306951523);
Epoch 3: D (0.2830143868923187 real_err, 0.25805848836898804 fake_err) G_l (0.02052653394639492 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.029447432607412338);
Epoch 4: D (0.08148318529129028 real_err, 0.15486261248588562 fake_err) G_l (0.0037686177529394627 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.09675496816635132);
Epoch 5: D (0.6273917555809021 real_err, 0.5055474638938904 fake_err) G_l (0.02917487919330597 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.0302715

KeyboardInterrupt: 